## Generate labels and translations

One the field mapping is done, this can be run to generate new csv entries for translations.

In [ ]:
import { authenticate } from './authentication.ts';
import { GATEWAY } from './helpers/routes.ts';

const token = await authenticate(GATEWAY, 'k.mweene', 'test');

In [ ]:
import { fetchEvents, fetchForms } from "./helpers/formsHandlers.ts";

const forms = await fetchForms(token)
const events = await fetchEvents(token)

In [ ]:
import { extractFieldType, extractFormFields } from './helpers/utils.ts'
import {
  DEFAULT_FIELD_MAPPINGS,
  MAPPING_FOR_CUSTOM_FIELDS,
} from './helpers/defaultMappings.ts'
import { COUNTRY_FIELD_MAPPINGS } from './helpers/countryMappings.ts'

const allMappings = {
  ...DEFAULT_FIELD_MAPPINGS,
  ...MAPPING_FOR_CUSTOM_FIELDS,
  ...COUNTRY_FIELD_MAPPINGS,
}

const birthFormFields = extractFormFields(forms, 'birth')
const deathFormFields = extractFormFields(forms, 'death')

const birthEvent = events.find((x) => x.id === 'v2.birth')
const deathEvent = events.find((x) => x.id === 'v2.death')

const birthEventFields = new Set(
  extractFieldType(birthEvent, 'fields').filter((x) => x)
)

const deathEventFields = new Set(
  extractFieldType(deathEvent, 'fields').filter((x) => x)
)

const formFields = [...birthFormFields, ...deathFormFields]
const eventFields = [...birthEventFields, ...deathEventFields]

const labelMaps = {}

const isCountrySelect = (field) => {
  return field.options?.resource === 'countries'
}

Object.keys(allMappings).forEach((k) => {
  const v1 = formFields.find((f) => f.id === k)
  const v2 = eventFields.find((e) => e.id === allMappings[k])
  if (v1 && v2) {
    labelMaps[v2.label.id] = v1.label.id
    if (v1.options && v2.options && !isCountrySelect(v1)) {
      v2.options.forEach(o => {
        const match = v1.options.find(opt => opt.value === o.value)
        if(match){
          labelMaps[o.label.id] = match.label.id
        }
      })
    }
    if (v1.options && v2.options && !isCountrySelect(v1)) {
      v2.options.forEach(o => {
        const match = v1.options.find(opt => opt.value === o.value)
        if(match){
          labelMaps[o.label.id] = match.label.id
        }
      })
    }
  }
})

labelMaps


In [ ]:
import { parse, stringify } from 'jsr:@std/csv'

const PATH_TO_CLIENT_CSV =
  '../../opencrvs-countryconfig/src/translations/client.csv'

const translations = await Deno.readTextFile(PATH_TO_CLIENT_CSV)

const transJson = parse(translations, {
  skipFirstRow: true,
  strip: true,
})

const newTrans = []

for (const [key, value] of Object.entries(labelMaps)) {
  const match = transJson.find((t) => t.id === value)
  newTrans.push({ ...match, id: key })
}

stringify(newTrans, { columns: Object.keys(newTrans[0]) })
